## 24. Каскадные операции

### 🧷 Каскадные операции

**Постановка проблемы:**  
Если удалить родителя, но не удалить зависимые строки — в базе останется "мусор". Или наоборот — нельзя будет удалить из-за ограничений. Каскадные операции (`ON DELETE CASCADE`) решают эту проблему.

**Краткая теория:**  
`CASCADE` означает, что при удалении или обновлении строки в родительской таблице автоматически изменяются/удаляются строки в дочерней.

- `ON DELETE CASCADE` — удалить все дочерние строки
- `ON UPDATE CASCADE` — обновить внешний ключ в дочерней таблице

Применяется в `FOREIGN KEY` при создании таблицы.

**Пример:**

In [ ]:
# Каскадное удаление в SQLite
import sqlite3
conn = sqlite3.connect(':memory:')
conn.execute("PRAGMA foreign_keys = ON")  # Включаем поддержку внешних ключей
cursor = conn.cursor()

# Родительская таблица
cursor.execute('''CREATE TABLE authors (
    id INTEGER PRIMARY KEY,
    name TEXT
)''')
# Дочерняя таблица с каскадом
cursor.execute('''CREATE TABLE books (
    id INTEGER PRIMARY KEY,
    title TEXT,
    author_id INTEGER,
    FOREIGN KEY (author_id) REFERENCES authors(id) ON DELETE CASCADE
)''')
# Добавим автора и книги
cursor.execute("INSERT INTO authors (name) VALUES ('Толстой')")
author_id = cursor.lastrowid
cursor.executemany("INSERT INTO books (title, author_id) VALUES (?, ?)", [
    ("Война и мир", author_id),
    ("Анна Каренина", author_id)
])
conn.commit()
# Удаляем автора
cursor.execute("DELETE FROM authors WHERE id = ?", (author_id,))
conn.commit()
# Проверяем, удалились ли книги
cursor.execute("SELECT * FROM books")
print("📕 Книги после удаления автора:")
print(cursor.fetchall())